In [1]:
from IPython.display import IFrame
from docling.document_converter import DocumentConverter
import boto3
import os
from sdg_hub.core.flow import FlowRegistry
from sdg_hub.core.blocks import BlockRegistry
import pypdfium2 as pdfium
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import LanceDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_graph_retriever import GraphRetriever
from langchain_core.documents import Document
from lancedb.rerankers import LinearCombinationReranker
from lancedb.embeddings import get_registry
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import lancedb
from huggingface_hub import snapshot_download
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, SentenceTransformerEmbeddings
from transformers import AutoTokenizer
from enum import Enum
import traceback
import re
import pandas as pd
from graph_retriever.strategies import Eager, Mmr
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from langchain_experimental.text_splitter import SemanticChunker
from pandas import option_context

In [2]:
endpoint_url = os.getenv('AWS_S3_ENDPOINT')
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket = os.getenv("AWS_S3_BUCKET")

llm = ChatOpenAI(
    model="openai/gpt-oss-20b", # os.getenv('QWEN25CODER_MODEL_ID'),
    api_key=os.getenv('OPENROUTER_TOKEN'),
    base_url=os.getenv('OPENROUTER_API_BASE'),
    temperature=0.1,
)

embedding_model = SentenceTransformerEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", 
    model_kwargs={"trust_remote_code":True
})

vectorstore_connection = lancedb.connect(f"s3://data/lancedb-graphrag",
    storage_options={
        "endpoint_url": endpoint_url,
        "aws_access_key_id": access_key_id,
        "aws_secret_access_key": secret_access_key,
        "s3_force_path_style": "true",
        "allow_http": "true",
    }
)

vectorstore = LanceDB(
    mode="append",
    embedding=embedding_model,
    connection=vectorstore_connection,
)

/tmp/ipykernel_37910/3974311152.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(
2025-10-23 13:52:34,442 - INFO - Use pytorch device_name: cuda:0
2025-10-23 13:52:34,443 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


In [3]:
# Cleanup
table = vectorstore_connection.open_table('vectorstore')

In [4]:
df = table.to_pandas()
df = pd.read_json(df.to_json())

/tmp/ipykernel_37910/1541054210.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df.to_json())


In [5]:
pd.set_option('display.max_colwidth', 200)
df

,vector,id,text,metadata
0,"[-0.0342968889, -0.042373742900000004, 0.0378167368, -0.0260634497, -0.0448614024, 0.0093835294, 0.0015629994000000002, -0.0134432903, 0.0451927595, -0.0248189103, -0.002079342, -0.0298561715, 0.0...",8bb24541-fefe-45df-b7df-37f60c6d2928,## CHAPTER 1,"{'Header 2': 'CHAPTER 1', 'keywords': ['Chapter', 'Section', 'Title', 'Heading', 'Document', 'Content', 'Outline', 'Intro', 'Narrative', 'Structure'], 'parent': '3c62835b5d40374ebf90d3e0d1d364929d..."
1,"[-0.0750388354, -0.050603427000000006, 0.0107633881, 0.0058810315, 0.0162374619, -0.0014325839, 0.0169347171, 0.0018467823, -0.088059783, -0.07826475050000001, 0.0165682118, 0.0240742546, 0.036464...",41dcdf8e-8d28-4e11-aa41-44d2d50d5875,"## Introduction to ColdFusion MX \nThis chapter describes ColdFusion MX and the role it plays in Internet applications, including Flash MX based applications. It also provides an introduction to ...","{'Header 2': 'Introduction to ColdFusion MX', 'keywords': ['ColdFusion', 'MX', 'Internet', 'Applications', 'Flash', 'Chapter', 'Role', 'Introduction', 'Topics', 'Book'], 'parent': 'c91c07160eb3d0f..."
2,"[-0.0719235614, -0.0332136564, -0.0208625365, -0.0345157571, 0.025180548400000002, -0.0039859028, -0.039131388100000004, -0.0223666318, -0.0407673828, -0.0286668986, 0.0175216887, 0.0072012134, 0....",d87ececc-7a4c-42f9-9c3f-7892ea503773,## Contents \n| • About Internet applications and web application servers.......................................... | 2 |\n|-------------------------------------------------------...,"{'Header 2': 'Contents', 'keywords': ['Internet', 'Applications', 'Web', 'Servers', 'ColdFusion', 'MX', 'Flash', 'J2EE', 'Architecture', 'Features'], 'parent': '111dbcc05d02b07b67ddb92972b42a4568f..."
3,"[-0.0523346923, -0.0558563136, -0.0119325221, -0.0378616899, 0.0172001477, -0.0045358995, 0.0176018216, 0.0161718242, -0.10841531310000001, -0.0764516518, 0.0322018415, 0.0214567427, 0.0407131575,...",54d9e8d9-043b-4d30-824e-927476b8b3cd,"## About Internet applications and web application servers \nWith ColdFusion MX, you develop Internet applications that run on web application servers. The following sections introduce Internet a...","{'Header 2': 'About Internet applications and web application servers', 'keywords': ['ColdFusion', 'MX', 'Internet', 'applications', 'web', 'servers', 'develop', 'environment', 'role', 'sections']..."
4,"[-0.0683347285, -0.023014098400000002, 0.0058056847, -0.021464256600000002, -0.0047181351000000005, 0.0179477967, -0.0367931835, 0.0007107656, -0.0504815802, -0.07145467400000001, 0.0271896366, 0....",a635b0ee-dea3-4675-909a-895f9eea6d26,"## About web pages and Internet applications \nThe Internet has evolved from a collection of static HTML pages to an application deployment platform. First, the Internet changed from consisting o...","{'Header 2': 'About web pages and Internet applications', 'keywords': ['Dynamic', 'E-commerce', 'Integration', 'XML', 'Webservices', 'J2EE', '.NET', 'ColdFusion', 'Development', 'Infrastructure'],..."
...,...,...,...,...
9273,"[-0.0633130819, -0.0238181241, 0.009058604000000001, -0.0148569448, 0.0585586242, 0.0181419328, 0.0042345594, 0.0004215855, 0.0026154397, -0.018677353900000002, -0.0392003171, 0.0083987759, 0.0108...",0ec9e4b9-ef2a-4d82-918d-9183f037a1a3,"***Topics***:\n1. **Conditional Logic in CFML** – `cfif`, `cfelse` statements and their syntax. \n2. **Form Scope Handling** – accessing and manipulating `form` variables. \n3. **Using `isDefine...","{'Header 2': 'To create an update page with cfquery:', 'keywords': ['Conditional', 'Form', 'Scope', 'Validation', 'Security', 'Debugging', 'Organization', 'Defaults', 'Workflow', 'Sanitization'], ..."
9274,"[-0.0783785582, -0.0371962078, -0.004517134700000001, 0.0020850587, 0.072000958, 0.0034436747, -0.0059126788000000005, 0.0083623892, 0.0187443253, -0.0188538376, 0.0057816925, -0.00448542, -0.0093...",1a184008-a116-4dc7-8609-fb7178221cbd,***Components***:\n**1. HTML Structure** \n - `<htm

In [6]:
# pd.set_option('display.max_colwidth', None)
# df[df["text"].str.contains("The following example shows a simple conditional loop")].iloc[0]
# df.iloc[19669]['text']
pd.set_option('display.max_colwidth', 200)

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
query = "Provide a comprehensive overview of how to create a list of elements in Coldfusion."
relevant_docs = retriever.invoke(query)

In [8]:
# Perform some data preprocessing on the results

def unique_documents(documents):
    unique_docs = []
    seen_page_contents = set()
    for doc in documents:
        if doc.page_content not in seen_page_contents:
            unique_docs.append(doc)
            seen_page_contents.add(doc.page_content)
    return unique_docs

relevant_docs = unique_documents(relevant_docs)

for doc in relevant_docs:
    print(doc)

page_content='## Adding elements to an array  
You can add an element to an array by assigning the element a value or by using a ColdFusion function.' metadata={'Header 2': 'Adding elements to an array', 'keywords': ['array', 'element', 'add', 'assign', 'value', 'ColdFusion', 'function', 'syntax', 'operation', 'method'], 'parent': 'f673f9aee107d6e5529cbd1493f1d83895ac7b36e9e9721cb0dc06238cd49b71fe85f862', 'source': 'markdown/10_Developing_Apps_coldfusion.md'}
page_content='## Elements of a ColdFusion application  
Before you develop a ColdFusion application, you must determine how to structure the application and how to handle application-wide needs and issues. In particular, you must consider all of the following:  
- · The overall application framework
- · Application-level settings and functions
- · Reusable application elements
- · Shared variables
- · Application security and user identification  
The following sections introduce these application elements and provide references t

In [9]:
# Get Memory Usage
df.memory_usage().sum()

371120